In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
from glob import glob
import shutil
import math
import json
import subprocess

In [2]:
mp_drawing = mp.solutions.drawing_utils #Drawing utilities
mp_holistic = mp.solutions.holistic   #Holistic model

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) #color conversion
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

In [4]:
def draw_landmark(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS)
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

In [5]:
def draw_styled_landmark(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS,
                                mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1))
    
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))
    
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2))
    
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2))

In [ ]:
cap = cv2.VideoCapture(0) 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        
        # Read the frame
        ret, frame = cap.read()
        
        
        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        
        # Drawing landmarks
        draw_styled_landmark(image, results)
        
        cv2.imshow("OpenCV Feed", image)

        if cv2.waitKey(10) and 0xFF == ord("q"):
            break

    cap.release()
    cv2.destroyAllWindows()

In [ ]:
cap.release()
cv2.destroyAllWindows()

In [ ]:
results.right_hand_landmarks

In [ ]:
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

In [ ]:
pose = []
for res in results.pose_landmarks.landmark:
    test = np.array([res.x, res.y, res.z, res.visibility])
    pose.append(test)

In [ ]:
len(pose)

In [ ]:
pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)

In [ ]:
len(pose)

In [ ]:
pose.shape

In [ ]:
pose.flatten()

In [ ]:
lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)

In [ ]:
lh

In [ ]:
lh.shape

In [ ]:
np.zeros(21*3)

In [ ]:
rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)

In [ ]:
rh

In [ ]:
rh.shape

In [ ]:
results.face_landmarks.landmark

In [ ]:
face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)

In [ ]:
face

In [ ]:
face.shape

In [6]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [7]:
def blank_keypoints():
    pose = np.zeros(132)
    face = np.zeros(1404)
    lh = np.zeros(21*3)
    rh = np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [ ]:
extract_keypoints(results).shape

# Setup for collecting data

In [8]:
# Path for exported data, numpy array
Data_Path = os.path.join("MP_Data")

#Actions that we try to detect
actions = np.array(["hello", "thanks", "iloveyou"])

#Thirty videos worth of data7
no_sequence = 30

#Videos are going to be 30 frames
sequence_length = 30

## Create a folder for each action for recording its value

In [ ]:
for action in actions:
    for sequence in range(no_sequence):
        try:
            os.makedirs(os.path.join(Data_Path, action, str(sequence)))
        except:
            pass

In [ ]:
glob("Data/*")

In [10]:
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)

In [11]:
def makedirbyname(path):
    try:
        os.mkdir(path)
    except:
        shutil.rmtree(path)
        os.mkdir(path)

In [12]:
frame_width, frame_height = (256,256)

In [13]:
num_frame_per_video = 100

In [ ]:
words = glob("Data/*")
makedirbyname("Data_Numpy")
for word in words:
    videos = glob(word + "/*")
    for video in videos:
        idx = 1
        video_path = "/".join("".join(video.split(".")[0:-1]).split("/")[1:])
        createFolder(f"Data_Numpy/{video_path}")
        print(f"Processing collection for... Data_Numpy/{video_path}")
        try:
            cap = cv2.VideoCapture(video)
            video_frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            print(f"Number of frame = {video_frame_count}")
            with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
                while True:
                    ret, frame = cap.read()

                    if ret == False:
                        cap.release()
                        break
                        
                    #Resize the frame
                    resize_frame = cv2.resize(frame, (frame_width, frame_height))
                    
                    # Make detections
                    image, results = mediapipe_detection(resize_frame, holistic)
                    
                    # Drawing landmarks
                    draw_styled_landmark(image, results)
                    
                    #Extracting keypoints
                    keypoints = extract_keypoints(results)
                    np.save(f"Data_Numpy/{video_path}/{idx}", keypoints)
                    
                    if idx == video_frame_count and video_frame_count < num_frame_per_video:
                        idx+=1
                        while idx<=num_frame_per_video:
                            keypoints_blank = blank_keypoints()
                            np.save(f"Data_Numpy/{video_path}/{idx}", keypoints_blank)
                            idx+=1
                        break
                    elif idx == num_frame_per_video and video_frame_count > num_frame_per_video:
                        break

                    #cv2.imshow(video, image)
                    #print(f"Writing... Data_Numpy/{video_path}/{idx}.png")
                    #cv2.imwrite(f"Data_Numpy/{video_path}/{idx}.png", image)
                    idx+=1

                    if cv2.waitKey(10) and 0xFF == ord("q"):
                        break

                cap.release()
                #cv2.destroyAllWindows()
        finally:
            cap.release()
            if idx < num_frame_per_video:
                if os.path.exists(f"Data_Numpy/{video_path}/{idx}.npy"):
                    idx+=1
                while idx<=num_frame_per_video:
                    keypoints_blank = blank_keypoints()
                    np.save(f"Data_Numpy/{video_path}/{idx}", keypoints_blank)
                    idx+=1
                
            #cv2.destroyAllWindows()

In [ ]:
cap.release()
#cv2.destroyAllWindows()

In [ ]:
cv2.imwrite(f"{idx}.png", frame)

In [ ]:
video

In [ ]:
"/".join("".join(video.split(".")[0:-1]).split("/")[1:])

In [ ]:
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

In [ ]:
actions = ["computer", "book", "other"] # Ecrire un script qui récupère tout ça du fichier JSOn

In [21]:
actions = get_actions(content)

In [22]:
len(actions)

100

In [23]:
from tensorflow.keras.utils import to_categorical

2022-04-04 19:34:38.988747: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/romerik/.local/lib/python3.9/site-packages/cv2/../../lib64:
2022-04-04 19:34:38.988822: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [24]:
label_map = {label:num for num,label in enumerate(actions)}

In [ ]:
label_map

In [25]:
def get_data(train_percent=0.90):
    train_sequences, train_labels = [], []
    test_sequences, test_labels = [], []
    words = glob("Data_Numpy/*")
    for word in words:
        actual_label = word.split("/")[-1]
        videos = glob(word + "/*")
        #print(f"-------------{actual_label}-------")
        train_proportion = math.ceil(round(train_percent * len(videos)))
        for index,video_ in enumerate(videos):
            #print(f"********---> Video   {video_} ----- ")
            window = []
            for frame_num in range(1,num_frame_per_video+1):
                np_file = f"{video_}/{frame_num}.npy"
                #print(np_file)
                res = np.load(np_file)
                window.append(res)
            if index <= train_proportion:
                train_sequences.append(window)
                train_labels.append(label_map[actual_label])
            else:
                test_sequences.append(window)
                test_labels.append(label_map[actual_label])
                
    train_sequences = np.array(train_sequences)
    train_labels = np.array(train_labels)
    test_sequences = np.array(test_sequences)
    test_labels = np.array(test_labels)
                
    return train_sequences, train_labels, test_sequences, test_labels


In [ ]:
train_sequences, train_labels, test_sequences, test_labels = get_data(0.8)

In [ ]:
train_sequences.shape

In [ ]:
train_labels.shape

In [ ]:
test_sequences.shape

In [ ]:
test_labels.shape

In [ ]:
X = train_sequences

In [ ]:
X.shape

In [ ]:
y = to_categorical(train_labels).astype(int)

In [ ]:
y

In [ ]:
X_test = test_sequences

In [ ]:
X_test

In [ ]:
y_test = to_categorical(test_labels).astype(int)

### Sample Video Data by name(gloss)

In [ ]:
!ls Data/

In [ ]:
!ls WLASL/

In [15]:
def get_num_frame(filename):
    video_frame_count = 0
    try:
        cap = cv2.VideoCapture(filename)
        video_frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    except:
        pass
    finally:
        cap.release()
    return video_frame_count

In [ ]:
get_num_frame("WLASL/start_kit/videos/05747.mp4")

In [16]:
def get_duration(filename):
    result = subprocess.run(["ffprobe", "-v", "error", "-show_entries",
                             "format=duration", "-of",
                             "default=noprint_wrappers=1:nokey=1", filename],
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT)
    return float(result.stdout)

In [ ]:
get_duration("WLASL/start_kit/videos/05747.mp4")

In [ ]:
os.listdir("WLASL/start_kit/videos/")

In [ ]:
frames = []

In [ ]:
total_frame = 0
n=0
for i in os.listdir("WLASL/start_kit/videos/"):
    frames.append(get_num_frame(f"WLASL/start_kit/videos/{i}"))
    n+=1

In [ ]:
frames = np.array(frames)

In [ ]:
np.mean(frames)

In [ ]:
np.std(frames)

In [ ]:
!cat WLASL/start_kit/WLASL_v0.3.json  

In [20]:
with open("WLASL/start_kit/WLASL_v0.3.json", "r") as file:
    content = json.load(file)

In [ ]:
content

In [ ]:
os.path.exists("WLASL/start_kit/videos/00618.mp4")

In [17]:
def create_folders_video(content):
    for i in content:
        folder_path = f"Data/{i['gloss']}"
        createFolder(folder_path)
        for j in i['instances']:
            if os.path.exists(f"WLASL/start_kit/videos/{j['video_id']}.mp4"):
                shutil.copy(f"WLASL/start_kit/videos/{j['video_id']}.mp4", f"Data/{i['gloss']}/{j['video_id']}.mp4")

In [ ]:
create_folders_video(content)

In [18]:
def get_actions(content):
    actions = []
    for i in content:
        actions.append(i['gloss'])
    return actions

In [ ]:
npy_file = []

In [ ]:
for i in actions:
    try:
        for d in os.listdir(f"Data_Numpy/{i}"):
            npy_file.append(len(os.listdir(f"Data_Numpy/{i}/{d}")))
    except:
        pass

In [ ]:
len(npy_file)

In [ ]:
npy_file = np.array(npy_file)

In [ ]:
np.sum(npy_file==100)

In [ ]:
npy_file